In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, concatenate, BatchNormalization, Activation, Dropout
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 0.001, decay_steps = 10000, decay_rate = 0.9)
optimizer = tf.keras.optimizers.Adam(learning_rate = scheduler)

In [4]:
(x_train, y_train) , (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
def denseBlocks(x, growth_size, num_layers):
  for _ in range(num_layers):
    newBatch = BatchNormalization()(x)
    newBatch = Activation("relu")(x)
    newBatch  = Conv2D(growth_size, (3,3), padding = "same")(x)
    x = concatenate([x,newBatch])

  return x



def transitionBlocks(x):
  x = BatchNormalization()(x)
  x = Conv2D(32, (2,2), strides = 2)(x)
  return x

In [6]:
def build_model():
  inputs = Input(shape = (32,32,3))

  x  = Conv2D(32,(2,2))(inputs)
  x = denseBlocks(x,16,3)


  x  = transitionBlocks(x)

  x = denseBlocks(x,8,3)

  x = transitionBlocks(x)

  x = denseBlocks(x, 4,2)


  x = tf.keras.layers.GlobalAveragePooling2D()(x)

  x = tf.keras.layers.Dense(64)(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  x = Dropout(0.5)(x)
  x = tf.keras.layers.Dense(32)(x)
  x = BatchNormalization()(x)
  x = Activation("relu")(x)
  outputs = tf.keras.layers.Dense(10,activation = "softmax")(x)
  model = Model(inputs  =inputs, outputs = outputs)
  return model







In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)


model = build_model()
model.compile(optimizer = optimizer, loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)
assert  x_train.shape[0] == y_train.shape[0]
assert  x_val.shape[0] == y_val.shape[0]
history = model.fit(datagen.flow(x_train,y_train,batch_size = 128), epochs = 100, validation_data = (x_val,y_val), callbacks  = [EarlyStopping(monitor = "val_loss",min_delta = 0.01, patience = 10, verbose = 2, restore_best_weights = True)], verbose = 2)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
